<a href="https://colab.research.google.com/github/rabindra20-git/CS771_projects/blob/main/ass2_submitted.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab.patches import cv2_imshow
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import cv2
import numpy as np
import os
from joblib import Parallel, delayed
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

train_dir= '/content/drive/MyDrive/assn2/train'
reference_dir = '/content/drive/MyDrive/assn2/reference'

def vanish_lines(image, color_range=80):
    hsv_image = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    v = hsv_image[:,:,2]
    min_val = np.min(v)
    mask = cv2.inRange(v, np.array(min_val - color_range), np.array(min_val + color_range))
    result_image = cv2.bitwise_and(image, image, mask=mask)
    result_image[np.where(mask == 0)] = 255
    return result_image


def segment_last_letter_bw(image):

    # Convert the image to grayscale
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Apply thresholding to segment the image into binary form
    _, binary_image = cv2.threshold(gray_image, 127, 255, cv2.THRESH_BINARY_INV)

    # Find contours of the letters
    contours, _ = cv2.findContours(binary_image, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Sort the contours from left to right
    contours = sorted(contours, key=lambda c: cv2.boundingRect(c)[0])

    # Extract the last letter
    last_contour = contours[-1]
    x, y, w, h = cv2.boundingRect(last_contour)
    last_letter = gray_image[y:y+h, x:x+w]

    # Apply thresholding to convert to black and white
    _, last_letter_bw = cv2.threshold(last_letter, 127, 255, cv2.THRESH_BINARY)

    return last_letter_bw


# Process images using parallelization
def process_image(filename):
    if filename.endswith('.png'):
        img_path = os.path.join(train_dir, filename)
        img = cv2.imread(img_path)
        img = vanish_lines(img)
        img = segment_last_letter_bw(img)
        img = cv2.resize(img, (50, 50))
        return img

a = 1
files = sorted(os.listdir(train_dir), key=lambda x: int(x[:-4]) if x.endswith('.png') else a==1)

# Parallelize image processing using joblib
resized_images = Parallel(n_jobs=-1)(delayed(process_image)(filename) for filename in files)

# Filter out None values (if any) and convert to NumPy array
resized_images = np.array([img for img in resized_images if img is not None])

# Load labels
labels_file = os.path.join(train_dir, 'labels.txt')
with open(labels_file, 'r') as file:
    labels = [line.strip() for line in file.readlines()]

# Split the preprocessed images and labels into train and test datasets
X_train, X_test, y_train, y_test = train_test_split(resized_images, labels, test_size=0.2, random_state=42)

# Reshape the train and test data to 2D
X_train_2d = X_train.reshape(X_train.shape[0], -1)
X_test_2d = X_test.reshape(X_test.shape[0], -1)

# Create a logistic regression classifier and fit it to the training data
model = LogisticRegression()
model.fit(X_train_2d, y_train)

# Perform predictions on the test data
y_pred = model.predict(X_test_2d)

# Calculate the accuracy of the predictions
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

# Save the labels to a text file
output_file = os.path.join(train_dir, 'predicted_labels.txt')
with open(output_file, 'w') as file:
    for label in y_pred:
        file.write(label + '\n')

Accuracy: 1.0
